# Phylo2Vec - Demo

## Python

### Installation (pip)

In [2]:
!pip install phylo2vec

  Using cached numba-0.60.0-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (2.7 kB)
  Using cached numpy-1.23.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.3 kB)
  Using cached biopython-1.80-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (13 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached ete3-3.1.3-py3-none-any.whl
  Using cached llvmlite-0.43.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.8 kB)
Using cached biopython-1.80-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
Using cached numpy-1.23.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached numba-0.60.0-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (3.7 MB)
Using cached llvmlite-0.43.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (43.9 MB)
  Attempting uninstall: numpy
    F

### 3rd party imports

In [1]:
import numpy as np

from ete3 import Tree

### Core

#### Sampling a random tree

In [2]:
from phylo2vec.utils import sample, seed_everything

seed_everything(42)

sample?

Signature:       sample(n_leaves, ordered=False)
Call signature:  sample(*args, **kwargs)
Type:            CPUDispatcher
String form:     CPUDispatcher(<function sample at 0x7fdb50108b80>)
File:            ~/src/phylo2vec_dev/phylo2vec/utils/random.py
Docstring:      
Sample a random tree topology via Phylo2Vec

Parameters
----------
n_leaves : int
    Number of leaves
ordered : bool, optional
    If True, sample an ordered tree, by default False

    True:
    v_i in {0, 1, ..., i} for i in (0, n_leaves-1)

    False:
    v_i in {0, 1, ..., 2*i} for i in (0, n_leaves-1)

Returns
-------
numpy.ndarray
    Phylo2Vec vector
Class docstring:
Implementation of user-facing dispatcher objects (i.e. created using
the @jit decorator).
This is an abstract base class. Subclasses should define the targetdescr
class attribute.
Init docstring: 
Parameters
----------
py_func: function object to be compiled
locals: dict, optional
    Mapping of local variable names to Numba types.  Used to override
 

In [3]:
v = sample(n_leaves=10)

print(repr(v))

array([0, 2, 3, 1, 0, 7, 5, 1, 8], dtype=uint16)


In [4]:
v_ordered = sample(n_leaves=10, ordered=True)

print(repr(v_ordered))

array([0, 1, 1, 1, 2, 3, 0, 2, 0], dtype=uint16)


We can also check that a vector is valid

In [5]:
from phylo2vec.utils import check_v

check_v?

Signature: check_v(v)
Docstring:
Input validation of a Phylo2Vec vector

The input is checked to satisfy the Phylo2Vec constraints

Parameters
----------
v : numpy.ndarray
    Phylo2Vec vector
File:      ~/src/phylo2vec_dev/phylo2vec/utils/validation.py
Type:      function

In [6]:
check_v(v) # returns None

v_awkward = v.copy()

v_awkward[5] = 11

check_v(v_awkward) # AssertionError

[ 0  2  3  1  0 11  5  1  8] [ True  True  True  True  True  True  True  True  True] [ True  True  True  True  True False  True  True  True]


AssertionError: None

#### Converting a vector to a Newick string

* ```to_newick``` is a wrapper of two functions: ```_get_ancestry``` and ```_build_newick```

In [7]:
from phylo2vec.base import to_newick

to_newick?

Signature:       to_newick(v)
Call signature:  to_newick(*args, **kwargs)
Type:            CPUDispatcher
String form:     CPUDispatcher(<function to_newick at 0x7fdaaae744c0>)
File:            ~/src/phylo2vec_dev/phylo2vec/base/to_newick.py
Docstring:      
Recover a rooted tree (in Newick format) from a Phylo2Vec v

Parameters
----------
v : numpy.array
    Phylo2Vec vector

Returns
-------
newick : str
    Newick tree
Class docstring:
Implementation of user-facing dispatcher objects (i.e. created using
the @jit decorator).
This is an abstract base class. Subclasses should define the targetdescr
class attribute.
Init docstring: 
Parameters
----------
py_func: function object to be compiled
locals: dict, optional
    Mapping of local variable names to Numba types.  Used to override
    the types deduced by the type inference engine.
targetoptions: dict, optional
    Target-specific config options.
pipeline_class: type numba.compiler.CompilerBase
    The compiler pipeline type.

In [8]:
newick = to_newick(v)

newick

'((((0,(5,7)12)13,(((1,(8,9)10)11,4)14,6)15)16,3)17,2)18;'

In [12]:
from phylo2vec.base.to_newick import _get_ancestry

_get_ancestry?

Signature:       _get_ancestry(v)
Call signature:  _get_ancestry(*args, **kwargs)
Type:            CPUDispatcher
String form:     CPUDispatcher(<function _get_ancestry at 0x7f2ac2455d80>)
File:            ~/src/phylo2vec_dev/phylo2vec/base/to_newick.py
Docstring:      
Get the "ancestry" of v (see "Returns" paragraph)

v[i] = which BRANCH we do the pairing from

The initial situation looks like this:
                  R
                  |
                  | --> branch 2
                // \
  branch 0 <-- //   \  --> branch 1
               0     1

For v[1], we have 3 possible branches too choose from.
v[1] = 0 or 1 indicates that we branch out from branch 0 or 1, respectively.
The new branch yields leaf 2 (like in ordered trees)

v[1] = 2 is somewhat similar: we create a new branch from R that yields leaf 2

Parameters
----------
v : numpy.array
    Phylo2Vec vector

Returns
-------
ancestry : numpy.array
    Ancestry matrix
    1st column: child 1
    2nd column: child 2
    3rd c

In [13]:
ancestry = _get_ancestry(v)

ancestry

array([[ 2,  8, 10],
       [ 0,  1, 11],
       [11,  7, 12],
       [12, 10, 13],
       [13,  3, 14],
       [14,  4, 15],
       [15,  5, 16],
       [16,  6, 17],
       [17,  9, 18]], dtype=int16)

In [14]:
from phylo2vec.base.to_newick import _build_newick

_build_newick?

Signature:       _build_newick(ancestry)
Call signature:  _build_newick(*args, **kwargs)
Type:            CPUDispatcher
String form:     CPUDispatcher(<function _build_newick at 0x7f2ac2455fc0>)
File:            ~/src/phylo2vec_dev/phylo2vec/base/to_newick.py
Docstring:      
Build a Newick string from an "ancestry" array

The input should always be 3-dimensional with the following format:
1st column: child 1
2nd column: child 2
3rd column: parent node

The matrix is processed such that we iteratively write a Newick string
to describe the tree.

Parameters
----------
ancestry : numpy.ndarray
    "Ancestry" array of size (n_leaves - 1, 3)

Returns
-------
newick : str
    Newick string
Class docstring:
Implementation of user-facing dispatcher objects (i.e. created using
the @jit decorator).
This is an abstract base class. Subclasses should define the targetdescr
class attribute.
Init docstring: 
Parameters
----------
py_func: function object to be compiled
locals: dict, optional
    Map

In [15]:
newick_other = _build_newick(ancestry)

newick_other # same as newick1

'((((((((0,1)11,7)12,(2,8)10)13,3)14,4)15,5)16,6)17,9)18;'

For visualisation purposes, we can plot the tree using ete3

In [16]:
def plot_tree(newick):
    print(Tree(newick))

plot_tree(newick)


                        /-0
                     /-|
                  /-|   \-1
                 |  |
               /-|   \-7
              |  |
              |  |   /-2
            /-|   \-|
           |  |      \-8
         /-|  |
        |  |   \-3
      /-|  |
     |  |   \-4
   /-|  |
  |  |   \-5
--|  |
  |   \-6
  |
   \-9


#### Converting a Newick to a vector

* ```to_vector``` is a wrapper of three functions:
    * ```_reduce```: reduces a Newick string to an ancestry
    * ```_order_cherries```: orders all cherries according to their height (i.e., from leaf-level cherries to the root-level pairs). While traversing the cherry-list, we replace the parent by their largest descending leaf
    * ```_build_vector```: constructs a Phylo2Vec vector from cherry triplets
* ```to_vector_no_parents```: a wrapper to convert Newick strings without parent labels (see an example below). It also wraps three (similar) functions: ```_reduce_no_parents```, ```_order_cherries_no_parents```, ```_build_vector```.

In [17]:
from phylo2vec.base import to_vector

to_vector?

Signature: to_vector(newick)
Docstring:
Convert a Newick string with parent labels to a vector

Parameters
----------
newick : str
    Newick string for a tree

Returns
-------
v : numpy.ndarray
    Phylo2Vec vector
File:      ~/src/phylo2vec_dev/phylo2vec/base/to_vector.py
Type:      function

In [18]:
# Let's generate a new v with 7 leaves using sample
v7 = sample(7)

print(f"v (sampled): {repr(v7)}")

newick7 = to_newick(v7)

print(f"newick: {newick7}")

v7_new = to_vector(newick7)

print(f"v (convert): {repr(v7_new)}")

v (sampled): array([0, 1, 2, 5, 4, 2], dtype=uint16)
newick: (0,((1,((2,6)7,3)9)10,(4,5)8)11)12;
v (convert): array([0, 1, 2, 5, 4, 2], dtype=uint16)


In [ ]:
from phylo2vec.base.to_vector import _reduce, _find_cherries, _build_vector

ancestry7 = _reduce(newick7)

print("Ancestry: ")
print(repr(ancestry7), end="\n\n")

cherries7, _ = _find_cherries(ancestry7)

print("Cherries: ")
print(repr(cherries7), end="\n\n")

print(f"v (converted again): {repr(_build_vector(cherries7))}")

Ancestry: 
array([[ 2,  6,  7],
       [ 7,  3,  9],
       [ 1,  9, 10],
       [ 4,  5,  8],
       [10,  8, 11],
       [ 0, 11, 12]], dtype=int16)

Cherries: 
array([[2, 6, 6],
       [4, 5, 5],
       [2, 3, 3],
       [1, 2, 2],
       [1, 4, 4],
       [0, 1, 1]], dtype=int16)

v (converted again): array([0, 1, 2, 5, 4, 2], dtype=uint16)


* As explained previously, phylo2vec can also convert Newick strings without parent labels. We have several functions in ```phylo2vec.utils.newick``` to process Newick strings

In [21]:
from phylo2vec.utils import remove_parent_labels

newick7_no_parent = remove_parent_labels(newick7)

print(f"Newick with parent labels: {newick7}")
print(f"Newick with parent labels: {newick7_no_parent}")

Newick with parent labels: (0,((1,((2,6)7,3)9)10,(4,5)8)11)12;
Newick with parent labels: (0,((1,((2,6),3)),(4,5)));


In [22]:
from phylo2vec.base import to_vector_no_parents

to_vector_no_parents(newick7_no_parent) # Same as v7, v7_new, v7_new_new

array([0, 1, 2, 5, 4, 2], dtype=uint16)

### Matrix form

* Newick strings can also have branch lengths, so it is also desirable to store not only the topology (which the core Phylo2Vec does), but also the branch lengths

In this setup:
* 1st column is v[i]
* 2nd column is where leaf i branched out from branch v[i]
* 3rd column is the branch length leading to leaf i

Note: This part of the package lacks documentation, sorry about that!

In [ ]:
from phylo2vec.matrix import to_newick2 as to_newick_with_bls
from phylo2vec.matrix import to_matrix

from phylo2vec.utils import sample

In [24]:
# Let's sample another v

n_leaves = 5

v5 = sample(n_leaves)

print(repr(v5))

array([0, 1, 0, 3], dtype=uint16)


In [25]:
branch_lengths = np.random.uniform(low=0, high=1, size=(n_leaves - 1, 2)).round(6).astype(str)
print(repr(branch_lengths))

newick_with_bls = to_newick_with_bls(v5, bls=branch_lengths)

print(newick_with_bls)

array([['0.92716', '0.27232'],
       ['0.135558', '0.125388'],
       ['0.241659', '0.213101'],
       ['0.057425', '0.33182']], dtype='<U32')
((0:0.135558,(3:0.92716,4:0.27232)5:0.125388)6:0.057425,(1:0.241659,2:0.213101)7:0.33182)8;


### Metrics

We believe it is possible to implement a variety of metrics pertaining to trees using the Phylo2Vec.

These can be metrics between trees (we evoked calculating a Hamming distance between vectors in the Phylo2Vec paper), but also between leaves within a tree. An example of the latter is the [cophenetic distance](https://en.wikipedia.org/wiki/Cophenetic) (inspired by [ape](https://rdrr.io/cran/ape/man/cophenetic.phylo.html)).

In [27]:
from phylo2vec.metrics import pairwise_distances

In [30]:
v20 = sample(n_leaves=20)

print(repr(v20))

pairwise_distances(v, metric="cophenetic")

array([ 0,  0,  1,  3,  6,  8,  4, 13, 10,  1, 15,  9,  4, 19, 19, 11, 20,
       33, 26], dtype=uint16)


array([[0, 2, 5, 5, 6, 7, 8, 3, 5, 9],
       [2, 0, 5, 5, 6, 7, 8, 3, 5, 9],
       [5, 5, 0, 4, 5, 6, 7, 4, 2, 8],
       [5, 5, 4, 0, 3, 4, 5, 4, 4, 6],
       [6, 6, 5, 3, 0, 3, 4, 5, 5, 5],
       [7, 7, 6, 4, 3, 0, 3, 6, 6, 4],
       [8, 8, 7, 5, 4, 3, 0, 7, 7, 3],
       [3, 3, 4, 4, 5, 6, 7, 0, 4, 8],
       [5, 5, 2, 4, 5, 6, 7, 4, 0, 8],
       [9, 9, 8, 6, 5, 4, 3, 8, 8, 0]], dtype=uint32)

### Optimisation

In the Phylo2Vec paper, we showcased a hill-climbing optimisation scheme to demonstrate the potential of phylo2vec for maximum likelihood-based phylogenetic inference.

These optimisation schemes (to be written in ```opt```) are not thoroughly maintained as difficult to test. One notable goal is to integrate [GradME](https://github.com/Neclow/GradME) into phylo2vec

### Other utility functions

#### Finding the number of leaves in a Newick

In [19]:
from phylo2vec.utils import find_num_leaves

find_num_leaves?

Signature: find_num_leaves(newick)
Docstring:
Calculate the number of leaves in a tree from its Newick

Parameters
----------
newick : str
    Newick representation of a tree

Returns
-------
int
    Number of leaves
File:      ~/src/phylo2vec_dev/phylo2vec/utils/newick.py
Type:      function

In [20]:
find_num_leaves(newick7)

7

#### Removing and adding a leaf in a tree

One might want to prune or add nodes in an existing tree (a common example is the subtree-prune-and-regraft operation).

This is not a trivial operation as we need to re-compute the vector (as the number of leaves in the tree will have changed)

In [21]:
from phylo2vec.utils import remove_leaf

remove_leaf?

Signature:       remove_leaf(v, leaf)
Call signature:  remove_leaf(*args, **kwargs)
Type:            CPUDispatcher
String form:     CPUDispatcher(<function remove_leaf at 0x7efca9a98310>)
File:            ~/src/phylo2vec_dev/phylo2vec/utils/vector.py
Docstring:      
Remove a leaf from a Phylo2Vec v

Parameters
----------
v : numpy.ndarray
    Phylo2Vec vector
leaf : int
    A leaf node to remove

Returns
-------
v_sub : numpy.ndarray
    Phylo2Vec vector without `leaf`
sister : int
    Sister node of leaf
Class docstring:
Implementation of user-facing dispatcher objects (i.e. created using
the @jit decorator).
This is an abstract base class. Subclasses should define the targetdescr
class attribute.
Init docstring: 
Parameters
----------
py_func: function object to be compiled
locals: dict, optional
    Mapping of local variable names to Numba types.  Used to override
    the types deduced by the type inference engine.
targetoptions: dict, optional
    Target-specific config options.
p

In [22]:
leaf = 3

v6, sister_leaf = remove_leaf(v7, leaf=leaf)

In [23]:
plot_tree(newick7)
plot_tree(to_newick(v6))


               /-0
            /-|
           |   \-6
         /-|
        |  |   /-2
      /-|   \-|
     |  |      \-3
   /-|  |
  |  |   \-4
--|  |
  |   \-5
  |
   \-1

               /-0
            /-|
         /-|   \-5
        |  |
      /-|   \-2
     |  |
   /-|   \-3
  |  |
--|   \-4
  |
   \-1


In [24]:
from phylo2vec.utils import add_leaf

add_leaf?

Signature:       add_leaf(v, leaf, pos)
Call signature:  add_leaf(*args, **kwargs)
Type:            CPUDispatcher
String form:     CPUDispatcher(<function add_leaf at 0x7efca9a98550>)
File:            ~/src/phylo2vec_dev/phylo2vec/utils/vector.py
Docstring:      
Add a leaf to a Phylo2Vec vector v

Parameters
----------
v : numpy.ndarray
    Phylo2Vec vector
leaf : int >= 0
    A leaf node to add
pos : int >= 0
    A branch from where the leaf will be added

Returns
-------
v_add : numpy.ndarray
    Phylo2Vec vector including the new leaf
Class docstring:
Implementation of user-facing dispatcher objects (i.e. created using
the @jit decorator).
This is an abstract base class. Subclasses should define the targetdescr
class attribute.
Init docstring: 
Parameters
----------
py_func: function object to be compiled
locals: dict, optional
    Mapping of local variable names to Numba types.  Used to override
    the types deduced by the type inference engine.
targetoptions: dict, optional
    

In [25]:
# due to re-labelling in remove_leaf, we have to decrement sister_leaf
if sister_leaf >= leaf:
    sister_leaf -= 1

v_add = add_leaf(v6, leaf=3, pos=sister_leaf)

np.array_equal(v_add, v7)

True

#### Applying and create an integer mapping from a Newick string

* Newick strings usually do not contain integers but real-life taxa (e.g., animal species, languages...). So it is important to provide another layer of conversion, where we can take in a Newick with string taxa, and convert it to a Newick with integer taxa, with a unique integer --> taxon mapping.

In [28]:
n_leaves = 8

t = Tree()
t.populate(n_leaves)
nw_str = t.write(format=9)

print(nw_str)

print(t)

((aaaaaaaaae,(aaaaaaaaaf,(aaaaaaaaag,aaaaaaaaah))),(aaaaaaaaaa,(aaaaaaaaab,(aaaaaaaaac,aaaaaaaaad))));

      /-aaaaaaaaae
   /-|
  |  |   /-aaaaaaaaaf
  |   \-|
  |     |   /-aaaaaaaaag
--|      \-|
  |         \-aaaaaaaaah
  |
  |   /-aaaaaaaaaa
   \-|
     |   /-aaaaaaaaab
      \-|
        |   /-aaaaaaaaac
         \-|
            \-aaaaaaaaad


In [29]:
from phylo2vec.utils import create_label_mapping

create_label_mapping?

Signature: create_label_mapping(newick)
Docstring:
Create an integer-taxon label mapping (label_mapping)
from a string-based newick (where leaves are strings)
and produce a mapped integer-based newick (where leaves are integers)
this also remove annotations pertaining to parent nodes

Parameters
----------
newick : str
    Newick with string labels

Returns
-------
newick_int : str
    Newick with integer labels
label_mapping : Dict str --> str
    Mapping of leaf labels (integers converted to string) to taxa
File:      ~/src/phylo2vec_dev/phylo2vec/utils/newick.py
Type:      function

In [33]:
from pprint import pprint

nw_int, label_mapping = create_label_mapping(nw_str)

plot_tree(nw_int)

pprint(label_mapping)


      /-3
   /-|
  |  |   /-2
  |   \-|
  |     |   /-0
--|      \-|
  |         \-1
  |
  |   /-7
   \-|
     |   /-6
      \-|
        |   /-4
         \-|
            \-5
{'0': 'aaaaaaaaag',
 '1': 'aaaaaaaaah',
 '2': 'aaaaaaaaaf',
 '3': 'aaaaaaaaae',
 '4': 'aaaaaaaaac',
 '5': 'aaaaaaaaad',
 '6': 'aaaaaaaaab',
 '7': 'aaaaaaaaaa'}


* The reverse operation is ```apply_label_mapping```

In [34]:
from phylo2vec.utils import apply_label_mapping

apply_label_mapping?

Signature: apply_label_mapping(newick, label_mapping)
Docstring:
Apply an integer-taxon label mapping (label_mapping)
from a string-based newick (where leaves are strings)
and produce a mapped integer-based newick (where leaves are integers)

Parameters
----------
newick : str
    Newick with integer labels
label_mapping : Dict str --> str
    Mapping of leaf labels (integers converted to string) to taxa

Returns
-------
newick : str
    Newick with string labels
File:      ~/src/phylo2vec_dev/phylo2vec/utils/newick.py
Type:      function

In [35]:
new_nw_str = apply_label_mapping(nw_int, label_mapping)

new_nw_str == nw_str

True

#### Benchmarks

The benchmarks are standalone scripts. They require additional libraries such as matplotlib, pandas, rpy2, seaborn. They are not meant to be used by the "average user", but are useful for research.

They should be executable using:


```bash
python -m benchmarks.name_of_benchmark --arg1 --arg2
```

### Tests

In [1]:
!pytest phylo2vec

============================= test session starts ==============================
platform linux -- Python 3.10.10, pytest-8.3.3, pluggy-1.5.0
rootdir: /home/nclow23/src/phylo2vec_dev
configfile: pyproject.toml
collected 1569 items                                                           

phylo2vec/tests/test_metrics.py ........................................ [  2%]
.                                                                        [  2%]
phylo2vec/tests/test_utils.py .......................................... [  5%]
........................................................................ [  9%]
........................................................................ [ 14%]
........................................................................ [ 19%]
........................................................................ [ 23%]
........................................................................ [ 28%]
.....................................................................